In [1]:
import numpy as np
import pandas as pd
#import os
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
init_notebook_mode(connected = True)
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("../data/lor.csv")
df.head

<bound method NDFrame.head of                                                   lor
0   It is with pleasure that I am writing this let...
1   To undertake this task successfully, she must ...
2   Copy of www.scholarstrategy.com - Do not redis...
3   She is a driven, organized teacher who\ndevelo...
4   Stacy is a perceptive, sharp, quick individual...
5   It gives me immense pleasure to recommend Mr M...
6   I am extremely happy to recommend Ms. Student ...
7   I am pleased to recommend to you Mr. STUDENT N...
8   I am extremely happy to recommend Ms. Student ...
9   I am writing this letter of recommendation for...
10  I am contented to endorse this letter of recom...
11  I take immense pleasure to recommend Student N...
12  It is a nice thing to know that my student Mr....
13   It is with pleasure that I issue this letter ...
14  With immense pleasure, I am writing this refer...
15  To The Admission Committee: \n\nI am glad to e...
16  I'm writing to recommend Mary Thompson for the..

In [3]:
import nltk
def lemmatize_string(s):
    from nltk.stem import WordNetLemmatizer
    wnl = WordNetLemmatizer()
    
    list2 = nltk.word_tokenize(s)
    lemmatized_string = ' '.join([wnl.lemmatize(words) for words in list2])
    return lemmatized_string 

In [4]:
l=[]
vectorizer = TfidfVectorizer(stop_words='english')
for i in range(21):
    l.append(lemmatize_string(df.iloc[i]['lor'].lower()))
X = vectorizer.fit_transform(l)


In [5]:
true_k = 5
with open('lor_model_picle','rb') as f:
    model=pickle.load(f)

In [6]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
order_centroids

array([[ 681, 1140,   41, ...,  680,  679,    0],
       [1031,  647,  673, ...,  745,  744,    0],
       [1031,  717, 1035, ...,  736,  735,    0],
       [ 505,  648,  147, ...,  693,  692,    0],
       [ 697, 1031, 1140, ...,  607,  603,  585]], dtype=int64)

In [7]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :50]:
        print(' %s' % terms[ind])
    print("\n")

Cluster 0:
 mary
 wa
 administrative
 company
 addition
 position
 solution
 streaming
 technical
 xyz
 video
 team
 industry
 excellent
 new
 service
 application
 skill
 network
 approach
 customer
 ve
 asia
 south
 technology
 audio
 project
 dedication
 department
 problem
 consistently
 assistant
 impressed
 work
 international
 graduate
 program
 oriented
 data
 constantly
 example
 client
 writing
 idea
 make
 strongly
 expertise
 intern
 enterprise
 health


Cluster 1:
 student
 life
 man
 subject
 electrical
 master
 peer
 character
 university
 worthy
 feel
 ability
 make
 taught
 ha
 unique
 stand
 engineering
 young
 mr
 aid
 nature
 known
 aptitude
 hard
 hallmark
 quarter
 scholastic
 close
 true
 overall
 adequate
 reflection
 analyze
 having
 nice
 laurel
 enduring
 diligent
 accomplish
 proud
 devotion
 financial
 pursue
 inquisitive
 bring
 confidently
 institute
 deeper
 chosen


Cluster 2:
 student
 ms
 subject
 extremely
 skill
 considering
 situation
 field
 gradu

In [8]:
def predict_cluster(s):    
    X = vectorizer.transform([lemmatize_string(s)])
    predicted = model.predict(X)
    return predicted[0]

In [9]:
def get_distance(s,order_centroids,true_k):
    X = vectorizer.transform([lemmatize_string(s)])
    distance=np.zeros((X.shape[0], true_k))
    for k in range(true_k):
        row_norm = np.linalg.norm(X - order_centroids[k, :], axis=1)
        distance[:, k] = np.square(row_norm)
    return distance.tolist()[0]

In [10]:
def get_score(s,order_centroids,true_k):
    d=get_distance(s,order_centroids,true_k)
    s0=26/d[0]
    s1=24/d[1]
    s2=29/d[2]
    s3=30/d[3]
    s4=19/d[4]
    final_score=(s0+s1+s2+s3+s4)/5
    return final_score

In [11]:
# ll=[]
# scaler = MinMaxScaler()
# for i in range(20):
#     x=get_score(l[i],order_centroids,true_k)
#     ll.append(x)
    
# scaled = scaler.fit_transform(np.array(ll).reshape(-1, 1))
# for i in range(20):
#     print(scaled[i][0]*5)

In [12]:
def get_final_score(s,order_centroids,true_k):
    ll=[]
    scaler = MinMaxScaler()
    for i in range(20):
        x=get_score(l[i],order_centroids,true_k)
        ll.append(x)
    ll.append(get_score(s,order_centroids,true_k))
    scaled = scaler.fit_transform(np.array(ll).reshape(-1, 1))

    return scaled[len(ll)-1]*5


In [13]:
lor_str='It is a feeling of immense pleasure to recommend Ms. Krisha Mehta, a student of the computerscience department, for the graduate program at your university. She has a good scholastic recordthat shows constant improvement each year. I have known her for the past 3 years and would like togive an overview of her technical and interpersonal skills.I got acquainted to her while I taught her Object Oriented Methodology, Operating Systems andDistributed Databases. She has displayed her knowledge, both theoretically and practically in thesesubjects and scored 10 grade points. She shows immense interest during lectures. The doubts andquestions raised by her are insightful and are driven by her eagerness to learn. She walks that extramile to learn new things and apply those concepts in practical life.Krisha assisted me on my research that focuses on extracting user information from spatio-temporaldata. She preprocessed google maps based location data and later used data mining and analysis toabstract meaningful temporal and spatial content for users eventually finding patterns about them.Krisha is a resourceful student who comes up with different solutions for a given problem. Sheperformed methodically, ensuring she knew the work previously done in the field, comparing andevaluating the methods that have been adopted as well as keeping herself updated with the latestdevelopments in the domain. Krisha, over the course of three months, also acquired basicknowledge of the peripheral domains with a possibility of exploring new approaches along witbalancing her academia. This helped me see her passion for research. She displayed her proficiencyin Python as well as the ability to learn new concepts while working on this project. She is also mentor at “Learn IT, Girl," an initiative for women in technology and developed a JavaScript baseproject “What is Privilege” which is a good way of giving back to the community and encouragewomen to learn science and technology.Krisha got an opportunity to intern at Indian Institute of Technology, Bombay which is quite rarefor a sophomore student. While there, she got well versed with using machine learning techniquesto improve the accuracy of smart sensors used for weather prediction. She also developed machinelearning algorithms to generate user profiles based on current investment portfolio for a financialstartup named CodeBreak. During this, she learnt the serviceability and functionality of thetechnologies involved and its various real world applications. Both (the institute and the company]have given superlative feedback about her performance and professional etiquettes. Overall, she isan assiduous and determined individual who belongs in the top 5% of her class when it comes tobalancing her academics, co-curricular activities, research as well as her work experience.To conclude, I am confident that the graduate program at your university suits her ambitions. Shewill without doubt continue to improve and work with dedication at your university and wouldprove to be a great addition to your department. I recommend Krishas candidature to your graduateprogram, without any reservations.I am pleased to provide a letter of recommendation for Ms. Krisha Mehta who aspires to pursue her graduatestudies in computer science at your university. In my capacity as her teacher for computer networks I haveknown her for almost 2 years as a responsible and focused student.Ms. Mehtas assignments are thoroughly researched, neat and well-written, highlighting the difficultconcepts in a lucid way. She would be the first in her batch to complete the work assigned to her and wouldassist others who needed help. Overall, her knowledge about computer networks was impeccable, which wasreflected in her academic grades too. Ms. Mehta also displayed efficient problem solving skills during hercomputer networking internship at Indian Institute of Management, Ahmedabad, Indias top business school.She was a research intern there studying and working on different algorithms to make Ad Hoc On-DemandDistance Vector(AODV) routing more efficient. Owing to this interest of hers in computer networks, she isnow working on a project that uses different probabilistic broadcast schemes to reduce the amount ofoverhead experienced by wireless networks under my guidance. Krisha independently studied andimplemented a few probabilistic broadcast algorithms that she thought that were relevant to the project that iscurrently in its early stage of development. Krishas eagerness to learn new things as well as the zeal andperseverance with which she does her research work is notable.Her desire to gain hands-on training in resolving real-world challenges is evident through her projects. Idistinctly remember her discussion with me wherein she wanted to contribute through her technicalknowledge to the students in high school and hence she devised an innovative application which involvedanimated visualizations to explain concepts of chemistry and mathematics more intuitively. Currently, herproject on “Enhancement of Coastal Security by Detecting Anomalies in the Current Ship Trajectories” isunder review for a research grant by the Government of India.Krishas knowledge is not confined to classroom learning only. She was one of the few Indians selected toattend Deep Learning Indaba, an international machine learning summer school. This year she attended thecomputer vision summer school and the machine learning summer school at International Institute ofInformation Technology, Hyderabad. She was one of the very few undergraduate students to get selected forthese schools that were mainly aimed for postgraduate students. These schools gave her a platform to learnconcepts outside of the university curriculum as well as exposed her to the research community.Krishas clarity of thought and confidence with which she could explain complicated concepts weredemonstrated when she was one of the speaker at HackPrep, a web development seminar for students in ourcollege. My association with Krisha so far has convinced me that she is very genuine in pursuit of heracademic excellence and that her work speaks for itself. As a mentor at the student chapter DJ-Unicode, sheprovides guidance and training to students on web development. Her remarkable writing skills weredemonstrated when she served as the editor of national level student chapter Computer Society of India.I strongly recommend her for graduate school. I am confident that she will continue to be a promising,hard-working and competitive student. I wish her good luck for graduate studies at your university.'

In [14]:
lor_rating=get_final_score(lor_str,order_centroids,true_k)
print("LOR rating: "+str(lor_rating.tolist()[0]))

LOR rating: 2.668471817705722
